In [1]:
import os
from decouple import config
from dotenv import load_dotenv


In [2]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
OPENAI_API_KEY = config('OPENAI_API_KEY')



In [3]:
from langchain_openai import ChatOpenAI

In [4]:
LLM_CONFIG = {
    "api_key" : OPENAI_API_KEY, 
    "model" : "gpt-4o"
}

In [5]:
model = ChatOpenAI(**LLM_CONFIG)

- starting to promt messages to our openai model

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

In [7]:
messages = [
    SystemMessage(content="Translate from English to Spanish"),
    HumanMessage(content="hi!"),
]

In [8]:
result = model.invoke(messages)
print(result)

content='¡Hola!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 18, 'total_tokens': 21}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_df84d6bd70', 'finish_reason': 'stop', 'logprobs': None} id='run-4441ddf2-54cf-4289-bca0-6ea8c1934064-0' usage_metadata={'input_tokens': 18, 'output_tokens': 3, 'total_tokens': 21}


- debugging the response

In [9]:
from langchain_core.output_parsers import StrOutputParser

In [10]:
parser = StrOutputParser()
parser.invoke(result)

'¡Hola!'

In [11]:
chain = model | parser
chain.invoke(messages)

'¡Hola!'

### Using a Different model

lets try a claude model from Anthropic

In [12]:
from langchain_anthropic import ChatAnthropic

load_dotenv()

ANTHROPIC_API_KEY = config('ANTHROPIC_API_KEY')

claude_model = ChatAnthropic(model='claude-3-opus-20240229', api_key=ANTHROPIC_API_KEY)

In [13]:
result = claude_model.invoke(messages)
parser.invoke(result)

'¡Hola!'

## Using prompting Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate

In [16]:
system_prompt = "Translate the following into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input_text}")
        
    ]

)

In [20]:
chain = prompt_template | claude_model | parser
chain.invoke({"language": "Swahili", "input_text": "What do you like to eat?"})

'Here is the Swahili translation of "What do you like to eat?":\n\nUnapenda kula nini?'

In [32]:
chain = prompt_template | claude_model | parser
chain.invoke({"language": "Kikuyu", "input_text": "Peleka story zako za jaba na huko!"})

'Ira ruga rũgano rwaku rwa maundu ma nja kũu!'